In [1]:
library(readxl) ## Leer excel
library(tidyverse) ## Manipular base de datos
library(igraph) ## Grafos
library(expm) 
library(writexl)
library(ggraph)
library(tidygraph)  # para manipular grafos estilo tidy


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Adjuntando el paquete: 'igraph'


The following objects are masked from 'package:lubridate':

    %--%, union


The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union


The following objects are masked from 'package:purrr':

    compose, simplify


The following object is masked from 'package:tidyr':

    crossing


The following object is masked from 'package:tibble':

    as_data_frame


The following objects are masked from 'package:s

In [2]:
# Lee un archivo Excel ubicado en la ruta especificada
matriz <- read_excel(
    "C:/Users/cfutr/OneDrive/Escritorio/CICS/2025/Tesis/Objetivos de Aprendizaje/Modelo/datos/Raw_Data/Matriz_de_Adyacencia_Pensiones_v5.xlsx", col_names=TRUE)

In [3]:
# Toma el data frame `matriz` (cargado desde un archivo Excel previamente)
# y lo transforma en una matriz de adyacencia numérica.

matriz_adj<-matriz%>%
  column_to_rownames(var = colnames(.)[1])%>% #Convierte la primera columna (por ejemplo, "Concepto") en los nombres de las filas.                         
  mutate(across(everything(), as.numeric)) %>%  #Aplica `as.numeric()` a todas las columnas del data frame.
  as.matrix() #Convierte el data frame limpio en una matriz

# Reemplaza todos los valores NA (valores perdidos) en la matriz de adyacencia `matriz_adj` por ceros.
matriz_adj[is.na(matriz_adj)] <- 0

# Crea un grafo dirigido (objeto de clase `igraph`) a partir de la matriz de adyacencia `matriz_adj`.
g <- graph_from_adjacency_matrix(matriz_adj, mode = "directed")

In [4]:
# Convertimos a data.frame las aristas
aristas <- as_data_frame(g, what = "edges")

# Generamos un identificador de cada par como "A--B" ordenado alfabéticamente
aristas$pair_id <- apply(aristas[, c("from", "to")], 1, function(x) paste(sort(x), collapse = "--"))

# Contamos cuántas veces aparece cada par (si es 2, hay doble dirección)
dobles <- aristas %>%
  group_by(pair_id) %>%
  filter(n() == 2) %>%
  ungroup()

# Resultado: lista de aristas que forman relaciones bidireccionales
dobles

from,to,pair_id
<chr>,<chr>,<chr>


In [ ]:
options(repr.plot.width = 30, repr.plot.height = 20)
# Convertir tu grafo igraph a tidygraph
grafo_tidy <- as_tbl_graph(g)

# Visualizar
ggraph(grafo_tidy, layout = "tree") +
  geom_edge_link(arrow = arrow(length = unit(1, 'mm')), end_cap = circle(2.5, 'mm')) +
  geom_node_point(size = 12, color = "steelblue") +
  geom_node_text(aes(label = name), repel = TRUE, size = 10, max.overlaps = Inf) +
  theme_void() +
  ggtitle("Precedencias sin redundancia")

In [ ]:
eliminar_aristas_redundantes <- function(g) {
  aristas <- as_data_frame(g, what = "edges")
  redundantes <- c()

  for (i in seq_len(nrow(aristas))) {
    from <- aristas$from[i]
    to   <- aristas$to[i]
    
    # Eliminar temporalmente la arista
    g_temp <- delete_edges(g, E(g, P = c(from, to)))

    # Comprobar si existe otra ruta de from a to
    caminos <- suppressWarnings(all_simple_paths(g_temp, from = from, to = to))
    
    # Solo marcar como redundante si hay otro camino alternativo
    if (length(caminos) > 0) {
      redundantes <- c(redundantes, i)
    }
    cat(sprintf("Revisando %d de %d: %s → %s\n", i, nrow(aristas), from, to))
  flush.console()
  }

  # Eliminar aristas redundantes del grafo original
  g_sin <- delete_edges(g, E(g)[redundantes])
  return(g_sin)
}


In [ ]:
g_sin_redundancia <- eliminar_aristas_redundantes(g)


In [ ]:
matriz_sin_redundancia <- as_adjacency_matrix(g_sin_redundancia, sparse = FALSE)

In [ ]:
# Crea un grafo dirigido a partir de la matriz de adyacencia sin redundancias.

g_sin_redundancia <- graph_from_adjacency_matrix(
  matriz_sin_redundancia,
  mode = "directed",
  diag = FALSE
)

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 15)
# Convertir tu grafo igraph a tidygraph
grafo_tidy <- as_tbl_graph(g_sin_redundancia)

# Visualizar
ggraph(grafo_tidy) +
  geom_edge_link(arrow = arrow(length = unit(4, 'mm')), end_cap = circle(3, 'mm')) +
  geom_node_point(size = 12, color = "steelblue") +
  geom_node_text(aes(label = name), repel = TRUE, size = 5, max.overlaps = Inf) +
  theme_void() +
  ggtitle("Precedencias sin redundancia")

In [ ]:
# Convertimos a data.frame y agregamos la columna con los nombres de fila
df_matriz <- as.data.frame(matriz_sin_redundancia)
df_matriz <- tibble::rownames_to_column(df_matriz, var = "Concepto")
write_xlsx(df_matriz, path = "C:/Users/cfutr/OneDrive/Escritorio/CICS/2025/Tesis/Objetivos de Aprendizaje/Modelo/datos/Intermedias/matriz_sin_redundancia_v2.xlsx")


In [ ]:
ggsave(
  filename = "C:/Users/cfutr/OneDrive/Escritorio/CICS/2025/Tesis/Objetivos de Aprendizaje/Modelo/Códigos/Plots/grafo_precedencias_v2.png",
  width = 30,
  height = 15,
  dpi = 300  # alta resolución
)

In [ ]:
# Lista para almacenar los caminos hacia atrás
caminos_hacia_atras <- list()

# Iterar sobre cada nodo del grafo
for (v in V(g_sin_redundancia)) {
  nombre_nodo <- V(g_sin_redundancia)$name[v]
  
  # Mostrar nodo que se está procesando
  cat(sprintf("🔄 Procesando caminos hacia: %s\n", nombre_nodo))
  flush.console()

  # Obtener todos los nodos desde los que se puede llegar a 'v'
  predecesores <- V(g_sin_redundancia)[suppressWarnings(subcomponent(g_sin_redundancia, v, mode = "in"))]
  
  # Obtener todos los caminos simples hacia 'v' desde cada predecesor
  caminos <- list()
  for (p in predecesores$name) {
    if (p != nombre_nodo) {
      caminos_desde_p <- suppressWarnings(all_simple_paths(g_sin_redundancia, from = p, to = nombre_nodo, mode = "out"))
      if (length(caminos_desde_p) > 0) {
        caminos[[p]] <- lapply(caminos_desde_p, function(path) V(g_sin_redundancia)$name[path])
      }
    }
  }

  # Guardar todos los caminos hacia el nodo actual
  caminos_hacia_atras[[nombre_nodo]] <- caminos
}

In [ ]:
# Versión detallada: 1 fila por camino
caminos_df <- do.call(rbind, lapply(names(caminos_hacia_atras), function(nodo) {
  caminos <- caminos_hacia_atras[[nodo]]
  if (length(caminos) == 0) {
    return(data.frame(
      nodo_objetivo = nodo,
      nodo_origen = NA,
      camino = NA,
      largo_camino = NA,
      stringsAsFactors = FALSE
    ))
  }
  do.call(rbind, lapply(names(caminos), function(origen) {
    lapply(caminos[[origen]], function(c) {
      data.frame(
        nodo_objetivo = nodo,
        nodo_origen = origen,
        camino = paste(c, collapse = " → "),
        largo_camino = length(c),
        stringsAsFactors = FALSE
      )
    }) %>% bind_rows()
  }))
}))


In [ ]:
caminos_df %>%
    filter(nodo_objetivo == "Opciones_Post_Oferta") %>%
    arrange(largo_camino)#%>%
    #distinct(nodo_origen)#%>%count()